# Explicit time schemes convergence order

In [ ]:
from trustutils import run

run.introduction("Elie Saikali and Yannick Gorsse.","29/04/2022")

# Description

We consider a classical 2D VDF conduction problem and we look to the order of convergence of 8 implemented Runge-Kutta schemes (low storage, rational and classical methods : RK2, RK2_classique, RRK2, RK3, RK3_classique, RK4, RK4_classique, RK4_classique_3_8), the two Adam Bashforth schemes and the Euler one. For each times scheme, we consider 5 calculations where the time step is fixed to 1, 0.1, 0.01, 0.001 and 0.0001 seconds respectively.


# Validation case

We consider a 1m x 1m domain with one cell, three symetrical bds and one Echange global at the top.
```
            Echange global
         +-------------------+
         |                   |
         |                   |
     Sym |                   | Sym
         |                   |
         |                   |
         +-------------------+
                  Sym
```

Assuming $\rho = cp = 1$.

$$
\int_{\Omega} \frac{\partial T}{\partial t} dv = - \int_{\Omega} D(\lambda , DT) dv
$$

$$
\frac{\partial T_i}{\partial t} = - \varepsilon \sum_{f} S_f (\lambda , DT)
$$

Expanding, we get:

$$
= 0 + 0 + 0 - h (T_i - T_{ext})
$$

Given $h=1$

Solving

$$
\frac{\partial T_i}{\partial t} + T_i = T_{ext}
$$

For a solution, let:

$$
T_i = C \Rightarrow C = T_{ext}
$$

General Solution

$$
T_i = K e^{-t} + T_{ext}
$$

When $t = 0$, then $100 = K + 50 \Rightarrow K = 50 = T_{ext}$.

So,

$$
T_i = T_{ext} e^{-t} + T_{ext}
$$



In [ ]:
run.TRUST_parameters("1.9.0")

In [ ]:
from trustutils import run 

run.reset()

schemes = [
            ("RK2", "runge_kutta_ordre_2"), 
            ("RK2_Classique", "runge_kutta_ordre_2_classique"), 
            ("RRK2", "runge_kutta_Rationnel_ordre_2"), 
            ("RK3", "runge_kutta_ordre_3"), 
            ("RK3_Classique", "runge_kutta_ordre_3_classique"),
            ("RK4", "runge_kutta_ordre_4"),
            ("RK4_Classique", "runge_kutta_ordre_4_classique"), 
            ("RK4_Classique_3_8", "runge_kutta_ordre_4_classique_3_8"),
            ("Euler", "schema_euler_explicite") , 
            ("AB2", "schema_adams_bashforth_order_2") ,
            ("AB3", "schema_adams_bashforth_order_3")
        ]

dt = [("1", "1.e0"),("1e_1", "1.e-1"), ("1e_2", "1.e-2"), ("1e_3", "1.e-3"), ("1e_4", "1.e-4")]

for i, sch in schemes:
    for j, dtmax in dt:
        run.addCaseFromTemplate("jdd.data", f"{i}/{j}", {"dtmax" : dtmax, "sch" : sch})

run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
run.executeScript("get_erreur")

# Convergence

In [ ]:
from trustutils import plot
import numpy as np
time_steps = [float(value) for _, value in dt]
x = np.linspace(0.0001,1,100)

a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
a.addPlot(0,f"Convergence order 1")
Eul = plot.loadText("erreur_Euler")
a.add(time_steps,Eul,label="Euler",marker='-o')
y = x*5
a.add(x,y,label="ordre 1", marker='--')
a.label("dt","erreur")
a.scale(xscale='log',yscale='log')

a.addPlot(1,f"Convergence order 2")
RK2 = plot.loadText("erreur_RK2")
RK2C = plot.loadText("erreur_RK2_Classique")
RRK2 = plot.loadText("erreur_RRK2")
AB2 = plot.loadText("erreur_AB2")
a.add(time_steps,RK2,label="RK2",marker='-o')
a.add(time_steps,RK2C,label="RK2_Classique",marker='-x')
a.add(time_steps,RRK2,label="RRK2",marker='-*')
a.add(time_steps,AB2,label="AB2",marker='-s')
y = x*x*1
a.add(x,y,label="ordre 2", marker='--')
a.label("dt","erreur")
a.scale(xscale='log',yscale='log')

a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
a.addPlot(0,f"Convergence order 3")
RK3 = plot.loadText("erreur_RK3")
RK3C = plot.loadText("erreur_RK3_Classique")
AB3 = plot.loadText("erreur_AB3")
a.add(time_steps,RK3,label="RK3",marker='-o')
a.add(time_steps,RK3C,label="RK3_Classique",marker='-x')
a.add(time_steps,AB3,label="AB3",marker='-s')
y = x*x*10
a.add(x,y,label="ordre 2", marker='--')
y = x*x*x*0.5
a.add(x,y,label="ordre 3", marker='--')
a.label("dt","erreur")
a.scale(xscale='log',yscale='log')

a.addPlot(1,f"Convergence order 4")
RK4 = plot.loadText("erreur_RK4")
RK4C = plot.loadText("erreur_RK4_Classique")
RK4C38 = plot.loadText("erreur_RK4_Classique_3_8")
a.add(time_steps,RK4,label="RK4",marker='-o')
a.add(time_steps,RK4C,label="RK4_Classique",marker='-x')
a.add(time_steps,RK4C38,label="RK4_Classique_3_8",marker='-*')
y = x*x*1
a.add(x,y,label="ordre 2", marker='--')
y = x*x*x*x*0.1
a.add(x,y,label="ordre 4", marker='--')
a.label("dt","erreur")
a.scale(xscale='log',yscale='log')

We note that the order of convergence is the same as the order of the schemes except for Adams Bashforth 3 and Low storage Runge-Kutta 4 where is order is only 2.